# Phenome-Wide analysis on COPDgene data: R PIC-SURE API use-case

This notebook is an illustration example of how to use the R **PIC-SURE API** to select and query data from an HPDS-hosted database. It takes as use-case a simple PheWAS analysis. This notebook is intentionally straightforward, and explanation provided are only aimed at guiding through the PheWAS analysis pipeline. For a more step-by-step introduction to the R PIC-SURE API, see the `R_PICSURE-API_101_PheWAS_example.ipynb` Notebook.

**Before running this notebook, please be sure to get an user-specific security token. For more information on how to proceed, see the `HPDS_connection.ipynb` notebook**

## Environment set-up

### System requirements
- R 3.5 or later

### Packages installation

In [ ]:
list_packages <- c("jsonlite", 
                   "ggplot2",
                   "plyr",
                   "dplyr",
                   "tidyr",
                   "purrr",
                   "stringr",
                   "ggrepel",
                   "devtools")

for (package in list_packages){
     if(! package %in% installed.packages()){
         install.packages(package, dependencies = TRUE, 
                         character.only = TRUE)
     }
     library(package, character.only = TRUE)
}

#### Installing latest R PIC-SURE API libraries from github

In [ ]:
devtools::install_github("hms-dbmi/pic-sure-r-client")
devtools::install_github("hms-dbmi/pic-sure-r-adapter-hpds")

#### Loading user-defined functions

In [1]:
source("R_lib/utils.R")


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




## Connecting to the a PIC-SURE network

In [4]:
PICSURE_network_URL <- "https://biodatacatalyst.integration.hms.harvard.edu/picsure"
resource_id <- "02e23f52-f354-4e8b-992c-d37c8b9ba140"
token_file <- "token.txt"

In [5]:
token <- TokenManager(token_file)

In [6]:
myconnection <- picsure::connect(url = PICSURE_network_URL,
                                 token = token)

In [7]:
resource <- hpds::get.resource(myconnection,
                               resourceUUID = resource_id)

## PheWAS analysis

In a nutshell, this PheWAS analysis follows those subsequent steps:
- Retrieving the variables dictionary, using the PIC-SURE API dedicated methods
- From the info provided by the dictionary, retrieving the data in an exploitable format through PIC-SURE API calls
- Data management
- Running univariate tests again every phenotypes variable
- Adjusting for multiple testing issue
- Plotting the results


This analysis is conducted using COPDGene Study data. The study overall goal is to detect underlying genetic factors to develop Chronic Obstructive Pulmonary Disease (COPD), and enrolled more than 10,000 individuals ([more information on COPDGene Study](http://www.copdgene.org)).

### 1. Retrieving variable dictionary from HPDS Database

In [8]:
all_variables <- hpds::find.in.dictionary(resource, "Genetic Epidemiology of COPD (COPDGene)")

In [9]:
variablesDict <- hpds::extract.dataframe(all_variables)
variablesDict <- variablesDict[order(variablesDict["name"]),]

In [11]:
# Display the variables tree hierarchy from the variables name
multiIndex <- get_multiIndex(variablesDict)
head(multiIndex)

depth 1,depth 2,depth 3,depth 4,depth 5,depth 6,name,simplified_name
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Genetic Epidemiology of COPD (COPDGene),Samples,Sample ID used in the Source Repository,NA,NA,NA,\Genetic Epidemiology of COPD (COPDGene)\Samples\Sample ID used in the Source Repository\,Sample ID used in the Source Repository
Genetic Epidemiology of COPD (COPDGene),Samples,Sample ID,NA,NA,NA,\Genetic Epidemiology of COPD (COPDGene)\Samples\Sample ID\,Sample ID
Genetic Epidemiology of COPD (COPDGene),Samples,Sample Use,NA,NA,NA,\Genetic Epidemiology of COPD (COPDGene)\Samples\Sample Use\,Sample Use
Genetic Epidemiology of COPD (COPDGene),Samples,Source repository where samples originate,NA,NA,NA,\Genetic Epidemiology of COPD (COPDGene)\Samples\Source repository where samples originate\,Source repository where samples originate
Genetic Epidemiology of COPD (COPDGene),Samples,Study,NA,NA,NA,\Genetic Epidemiology of COPD (COPDGene)\Samples\Study\,Study
Genetic Epidemiology of COPD (COPDGene),Samples,Subject ID,NA,NA,NA,\Genetic Epidemiology of COPD (COPDGene)\Samples\Subject ID\,Subject ID


### 2. Selecting variables and retrieving data from HPDS

All phenotypical variables available will be included in the analysis, except for the variables pertaining to the "Sub-study ESP LungGO COPDGene" category (very small and specific subpopulation of the COPDGene Study).

In [12]:
mask_pheno <- multiIndex["depth 2"] == 'Subject Phenotype'
mask_status <- multiIndex["depth 3"] == 'Affection status'
mask_vars <- mask_pheno | mask_status
selected_vars <- variablesDict[mask_vars, "name"]

In [16]:
my_query = hpds::new.query(resource = resource)
hpds::query.select.add(query = my_query, 
                      keys = selected_vars)
facts = hpds::query.run(query = my_query, result.type = "dataframe")

Warning message in if (has.key(key, self$data) == TRUE) {:
“la condition a une longueur > 1 et seul le premier élément est utilisé”
Warning message in if (str_detect(norm_str, "rs[0-9]+$")) {:
“la condition a une longueur > 1 et seul le premier élément est utilisé”
Warning message in if (str_detect(norm_str, "[0-9]+,[0-9\\.]+,.*")) {:
“la condition a une longueur > 1 et seul le premier élément est utilisé”


In [ ]:
str(selected_vars)

In [18]:
my_query = hpds::new.query(resource = resource)
hpds::query.anyof.add(query = my_query, 
                      keys = selected_vars)
facts = hpds::query.run(query = my_query, result.type = "dataframe")

Warning message in if (has.key(key, self$data) == TRUE) {:
“la condition a une longueur > 1 et seul le premier élément est utilisé”
Warning message in if (str_detect(norm_str, "rs[0-9]+$")) {:
“la condition a une longueur > 1 et seul le premier élément est utilisé”
Warning message in if (str_detect(norm_str, "[0-9]+,[0-9\\.]+,.*")) {:
“la condition a une longueur > 1 et seul le premier élément est utilisé”


ERROR: HTTP response was bad
Response [https://biodatacatalyst.integration.hms.harvard.edu/picsure/query/sync/]
  Date: 2019-12-12 16:11
  Status: 500
  Content-Type: application/json
  Size: 79 B



In [15]:
str(facts)

'data.frame':	320042 obs. of  2 variables:
 $ Patient.ID                                                                                : int  1 2 3 4 5 6 7 8 9 10 ...
 $ X.Genetic.Epidemiology.of.COPD..COPDGene..Subject.Phenotype.6MinWalk.Distance.walked..ft..: num  NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN ...


In [ ]:
variablesDict["name"]

### 3. Data-management

Since variable names are not the same between the dictionary and the dataframe columns, a temporary workaround is needed: parsing variables names from variables dictionary to match actual column names. (issue arise from the fact the R data.frame column names cannot contain spaces).

In [ ]:
variablesDict[["df_name"]] <- parsing_varNames(variablesDict[["name"]])
multiIndex[["df_name"]] <- variablesDict[["df_name"]]

In [ ]:
checking_parsing(names(facts)[-1], variablesDict[mask_vars, "df_name"])

#### Selecting variables regarding their types

One important step in a PheWAS is to get the distinction between categorical and numerical variables. This distinction is straightforward using the variables dictionary.

In [ ]:
mask_categories <- variablesDict[, "categorical"] == TRUE
categorical_varnames <- variablesDict[mask_categories & mask_vars, "df_name"]
continuous_varnames <- variablesDict[!mask_categories & mask_vars, "df_name"]

### Selecting the dependent variable to study
Most of PheWAS use a genetic variant as the variable used to separate the population between cases and controls. However the population doesn't have to be dichotomized using a genetic variant, and any phenotypic variable could be used to run a PheWAS analysis (see for example [*Neuraz et al.*, 2013](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1003405)). 

Here we will use the **COPD status** as the case-control variable to dichotomize the population in our analysis (ie the dependent variable for which univariate association test will be run against).

In [ ]:
dependent_var_name <- multiIndex[multiIndex[["simplified_name"]] == "00 Affection status",][["df_name"]]
categorical_varnames <- categorical_varnames[-which(categorical_varnames == dependent_var_name)]

In [ ]:
table(facts[[dependent_var_name]])

Then we subset our population regarding the relevant values for the COPD diagnosis variable (i.e. keeping "Case" and "Control" individuals, and discarding "Other", "Control, Exclusionary Disease", and null values).

In [ ]:
mask_dependent_var_name = facts[[dependent_var_name]] %in% c("Case", "Control")

In [ ]:
facts <- facts[mask_dependent_var_name,]

In [ ]:
count_case_control <- table(facts[[dependent_var_name]])

In [ ]:
sprintf("Control: %i individuals\nCase: %i individuals",
        count_case_control["Control"],
        count_case_control["Case"]) %>% cat()

### 4. Univariate statistical tests

Univariate logistic regression models will be runned using each phenotypical variable as the regressor, and Likelihood Ratio Test p-values for the model will be used to assess the significance of the association

In [ ]:
## Unified univariate tests
### Significance of association tests are retrieved from the p-values of a Likelihood ratio test
anova_model <- function(data, dependent_var, independent_var) {
    model <- glm(as.formula(paste(dependent_var, "~ 1 +", independent_var)),
                 data = data,
                 family = binomial(link="logit"))
    model_reduced <- glm(as.formula(paste(dependent_var, "~ 1")),
                         data = data,
                         family = binomial(link="logit"))
    p_val <- anova(model, model_reduced, test =  "LRT")[2, "Pr(>Chi)"]
    return(p_val)    
}

In [ ]:
independent_var_names = c(categorical_varnames, continuous_varnames)

In [ ]:
pvalues_list = list()
error_list =  list()
warning_list = list()
for (independent_var_name in independent_var_names) {
    data <- na.omit(facts[,c(dependent_var_name, independent_var_name)])
    tryCatch({
                pvalues_list[[independent_var_name]] <- anova_model(data, dependent_var_name, independent_var_name)
#                error_list[[independent_var_name]] <- NA
 #               warning_list[[independent_var_name]] <- NA
    },
             error = function(e) {
                print(paste("error", independent_var_name))
                pvalues_list[[independent_var_name]] <- NA
                error_list[[independent_var_name]] <- e
                warning_list[[independent_var_name]] <- NA
             },
             warning = function(w) {
                print(paste("warning", independent_var_name))

                pvalues_list[[independent_var_name]] <- NA
                error_list[[independent_var_name]] <- NA
                warning_list[[independent_var_name]] <- w                 
             }
        )
}

In [ ]:
df_pvalues <- data.frame(
    "df_name" = names(pvalues_list),
    "pvalues" = simplify(unname(pvalues_list)), 
    stringsAsFactors = F
)

In [ ]:
df_pvalues[["log_pvalues"]] <- -log10(df_pvalues$pvalues)

In [ ]:
multiIndex_enhanced <- dplyr::left_join(multiIndex, df_pvalues, by="df_name")
variablesDict_enhanced <- dplyr::left_join(variablesDict, df_pvalues, by="df_name")

In [ ]:
head(variablesDict_enhanced)

#### Univariate test p-values distribution

In [ ]:
ggplot(aes_string(x = "pvalues", 
                 fill = "categorical"),
       data = variablesDict_enhanced) +
geom_histogram(bins=20, position = "dodge") +
scale_fill_brewer(palette='Paired') + 
labs(title = "Distribution of non-adjusted p-values among tested phenotypes ", 
    subtitle = expression(italic("Likelihood Ratio Test"))) +
xlab("Unadjusted p-values") +
ylab("Count") +
theme_bw()

## 5. Multiple hypotheses testing correction: Bonferroni Method

In order to handle the multiple comparison issue (increase in the probability to "discover" false statistical associations, because of the number of tests performed), we will use the Bonferroni correction method. Although many other multiple comparison exist, Bonferroni is the most straightforward to use, because it doesn't require assumptions about variables correlation. Other PheWAS analysis also use False Discovery Rate controlling procedures ([see reference](https://en.wikipedia.org/wiki/False_discovery_rate)).

In a nutshell, Bonferonni allows to calculate a corrected "statistical significant threshold" according to the number of test performed. Every p-value below this threshold will be deemed statistically significant.

In [ ]:
variablesDict_enhanced$adj_pvalues <- p.adjust(variablesDict_enhanced$pvalues, method="bonferroni")

In [ ]:
variablesDict_enhanced$log_adj_pvalues <- -log10(variablesDict_enhanced$adj_pvalues)

## 6. Result visualisations: Manhattan plot

Manhattan plot is the classical results representation of a PheWAS analysis. It plots every each tested phenotypical variables on the X-axis, against its *-log(pvalue)* on the Y-axis. The horizontal line represent the adjusted significance level threshold.

#### Preparing data

In [ ]:
corrected_alpha <- 0.05/length(variablesDict_enhanced$pvalues) # Using Bonferonni method
adj_corrected_alpha <- -log10(corrected_alpha)

In [ ]:
non_nan <- which(!is.na(variablesDict_enhanced$pvalues))
plot_df <- multiIndex_enhanced[non_nan, ]
plot_df$df_name <- as.factor(plot_df$df_name)
plot_df$log_pvalues <- round(plot_df$log_pvalues, 5)

plot_df = multiIndex[, c("df_name", "depth 2")] %>% 
plyr::rename(replace = c("depth 2" = "category")) %>%
right_join(plot_df, by="df_name")

plot_df <- plot_df[order(plot_df$category),]
plot_df$category <- factor(plot_df$category)
plot_df$name <- factor(plot_df$name, levels=plot_df$name[order(plot_df$category)])

In [ ]:
# Suppressing Inf log(p-values)
plot_df <- plot_df[plot_df$log_pvalues != Inf,]

In [ ]:
# Selecting 4 largest p-values, to be annotated in the Manatthan plot

largest_pvalues_indices <- order(plot_df[["log_pvalues"]], decreasing=T)[1:4]
plot_df$to_annotate <- "no"
plot_df[largest_pvalues_indices, "to_annotate"] <- "yes"

#### Plotting the data

In [ ]:
options(repr.plot.width=12, repr.plot.height=8)

In [ ]:
my_theme <- theme_bw() + 
theme(axis.title.y = element_text(face="italic", size=15),
      title = element_text(size=20),
      axis.title.x = element_text(size=15),
      axis.text.x = element_text(angle=35, hjust=1),
      legend.position = "none",
      panel.grid.major.x = element_blank()
      )


In [ ]:
paired_colors <- c("navyblue", "lightskyblue")

In [ ]:
# Manatthan plot using geom_jitter --> more suitable when number of point per category is enough
ggplot(plot_df, aes(x=category, y=log_pvalues)) +
geom_jitter(alpha=1, aes_string(colour="category"), 
           width=0.5,
           size=3) +
geom_hline(yintercept=adj_corrected_alpha, linetype="dashed") +
scale_y_continuous(expand = c(0, 20) ) +
scale_color_manual(values = rep(paired_colors, times=20)) +
geom_label_repel( data=subset(plot_df, to_annotate=="yes"), aes(label=simplified_name), size=3.5) +
labs(title="Association between phenotypes variables and gene mutation (COPD status)", 
    x="Phenotypes", 
    y="- log10(p-values)",
    colour="Phenotypes categories") +
my_theme



Overall, it appears that most of the tested phenotypes covariates are above the adjusted threshold of significant association. However, it is not surprising at all, given the nature of our dependent variable: a lot of those variables are by nature tied directly to the COPD status. For instance, the 4 highest p-values (distance walked in feet, nebulizer for inhaled medication, too breathless to leave the house) are direct consequences of COPD disease.

This code can be used directly with any other variable present in the variable Dictionary. It only need to change the `dependent_var_name` value.